In [6]:
!git clone https://github.com/vincentnam/ProjetInterpromo2020G8.git

Clonage dans 'ProjetInterpromo2020G8'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 386 (delta 54), reused 80 (delta 25), pack-reused 277
Réception d'objets: 100% (386/386), 11.46 Mio | 1.00 Mio/s, fait.
Résolution des deltas: 100% (210/210), fait.


In [1]:
"""
Created on Fri Jan 3 13:28:13 CET 2019
Group 8
@authors: DANG Vincent-Nam
"""

from abc import ABCMeta, abstractmethod
from typing import Iterable

import numpy as np


# TODO :
#  - Tests unitaires et tests intégrations : test pipeline
#  (run_pipeline), levées d'erreur, etc...
#  - Traduire les commentaires en anglais (si besoin ?)
#  - Mettre à jour le pipeline pour prendre en compte des resultats
#    auxiliaires nécessaire pour le traitement suivant
#  - Gestion des hints plus formellement
#  - Gestion de l'héritage des docstrings



In [2]:
class NotProcessClass(Exception):
    def __init__(self, expression, message):
        """
        Classe d'exception pour indiquer qu'autre chose qu'un process a
        été rajouté dans le pipeline.
        Attributes:
            process_desc -- description du process
            process_class -- class du process
        """
        self.expression = expression
        self.message = message


In [3]:


# Metaclasse pour les processus
class MetaProcess(metaclass=ABCMeta):
    """
    Metaclasse pour la definition d'un processus. Permet de définir le
    comportement que doit avoir un processus pour fonctionner avec les
    pipeline de travail (classe Pipeline).
    """

    def check_attributes(self):
        """
        Attribut abstrait obligatoire permettant de définir une
        description du process effectué affiché si la verbosité est
        supérieure à 0.
        Cet attribut doit décrire ce qui est réalisée par le process
        et la librairie majoritairement utilisée pour réaliser ce
        process ainsi que la version de cette bibliothèque s'il y a.
        """
        if self.process_desc is None or self.process_desc == "":
            raise NotImplementedError("Définissez une description pour "
                                      + "le process.")

    def __init__(self, verbose=1):
        self.verbose = verbose
        self.check_attributes()
        super().__init__()
        if self.verbose > 0:
            print(self.__class__.__base__.__name__ + " : ", end=' ')
            print(self.process_desc)

    @property
    @abstractmethod
    def process_desc(self):
        """Name of the process to be able to identify the process"""
        return self.process_desc

    @abstractmethod
    def run(self, images: Iterable[Iterable]) -> None:
        """
        Réalise le traitement à effectuer sur la liste d'images. Ne
        retourne rien. Les modifications sont effectuées directement sur
        les images dans la liste.
        :param images: objet de array-like contenant les images à traiter
        """



In [4]:


# Classe abstraite pour les processus de pré-traitement de données
class Preprocess(MetaProcess):
    """
    Classe abstraite de définition d'un processus de pré-traitement des
    données.
    Pour définir un pré-traitement, il est nécessaire de définir une
    classe héritant de cette classe (Preprocess). Il doit
    obligatoirement être implémenté la fonction run(self,images) et
    l'attribut process_desc (avec une valeur différente de None ou "").
    Ex :
    class Preprocess_exemple(Proprocess):
        process_desc = "OpenCV4.0 -> data augmentation : rotations"
    ...
    """

    process_desc = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @abstractmethod
    def run(self, images: Iterable[Iterable]) -> None:
        pass


# Classe abstraite pour les processus de traitement de données
class Process(MetaProcess):
    """
    Classe abstraite de définition d'un processus de traitement des
    données.
    Pour définir un traitement, il est nécessaire de définir une
    classe héritant de cette classe (Process). Il doit
    obligatoirement être implémenté la fonction run(self,images) et
    l'attribut process_desc (avec une valeur différente de None ou "").
    Ex :
    class Preprocess_exemple(Proprocess):
        process_desc = "PyTesseract2.0-> recherche de caractères"
    ...
    """
    process_desc = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @abstractmethod
    def run(self, images: Iterable[Iterable], json: str) -> None:
        pass


# Classe abstraite pour les processus de post-traitement de données
class Postprocess(MetaProcess):
    """
    Classe abstraite de définition d'un processus de post-traitement des
    données.
    Pour définir un pré-traitement, il est nécessaire de définir une
    classe héritant de cette classe (Postprocess). Il doit
    obligatoirement être implémenté la fonction run(self,images) et
    l'attribut process_desc (avec une valeur différente de None ou "").
    Ex :
    class Preprocess_exemple(Proprocess):
        process_desc = "OpenCV4.0 -> alignement des predictions"
    ...
    """
    process_desc = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @abstractmethod
    def run(self, images: Iterable[Iterable]) -> None:
        pass

In [5]:


# Classe pour le pipeline
class Pipeline:
    """
    Classe permettant de définir un pipeline. Le pipeline execute dans
    l'ordre le pré_processing, le processing et le post_processing.
    Cette classe contient 3 numpy array contenant la liste des
    traitements pour chaque étapes à réaliser. Il faut utiliser les
    fonction add_pre_process
    """

    def __init__(self) -> None:
        self.pre_process: Iterable[Preprocess] = np.array([])
        self.process: Iterable[Process] = np.array([])
        self.post_process: Iterable[Postprocess] = np.array([])

    def add_processes(self, in_process: Iterable[MetaProcess]):
        """
        Ajoute une liste de process dans le pipeline.
        :param in_process: Iterable[MetaProcess] : Liste des process à
        ajouter au pipeline. Chaque
        """
        wrong_processes: tuple = ()
        for process in in_process:
            if not (isinstance(process, Postprocess)
                    or isinstance(process, Process)
                    or isinstance(process, Preprocess)):
                if MetaProcess in process.__class__.__mro__:
                    wrong_process = ((process.process_desc,
                                      process.__class__,),)

                    wrong_processes = wrong_processes + wrong_process

                else:
                    wrong_processes = wrong_processes + ((type(process),),)
                    continue

            else:
                if isinstance(process, Preprocess):
                    self.pre_process = np.append(self.pre_process,
                                                 np.array([process]))
                    print(process.process_desc + " a été ajouté.")
                if isinstance(process, Process):
                    self.process = np.append(self.process,
                                             np.array([process]))
                    print(process.process_desc + " a été ajouté.")

                if isinstance(process, Postprocess):
                    self.post_process = np.append(self.post_process,
                                                  np.array([process]))
                    print(process.process_desc + " a été ajouté.")
        if len(wrong_processes) > 0:
            raise NotProcessClass("Autre chose que des process ont "
                                  "été ajoutés au pipeline.", wrong_processes)

    def print_process(self):
        """
        Affiche les processus qui seront executés permettant ainsi de
        voir l'ordre d'execution des traitements dans le pipeline.
        """
        for process in self.pre_process:
            print(process.process_desc)
        for process in self.process:
            print(process.process_desc)
        for process in self.post_process:
            print(process.process_desc)

    # Pas besoin de retourner les variables : on modifie directement les images
    def run_pipeline(self, images: Iterable[Iterable]) -> None:
        """
        Execute le pipeline. Il sera executé dans l'ordre
            - le pré-processing
            - le processing
            - le post-processing
        Chaque process est conservé dans une liste et chaque groupe de
        process sera executé dans l'ordre dans lequel les process ont
        été ajoutés dans la liste de traitements.
        Les images sont directement modifiés.
        :param images: objet array-like : contient la liste de images
        :return: None
        """
        print("Début du pipeline : ")
        for num, pre_process in enumerate(self.pre_process):
            try:
                print("Doing : " + pre_process.process_desc)
                pre_process.run(images)
            except Exception as e:
                print("Le pré-processing numéro " + str(num)
                      + "( " + pre_process.process_desc
                      + " ) a levé une erreur.")
                print(e)

        for num, process in enumerate(self.process):
            try:
                print("Doing : " + process.process_desc)
                process.run(images)
            except Exception as e:
                print("Le processing numéro " + str(num)
                      + "( " + process.process_desc + " ) a levé une erreur.")
                print(e)

class BlackWhite(Preprocess):
    
        for num, post_process in enumerate(self.post_process):
            try:

                print("Doing : " + post_process.process_desc)
                post_process.run(images)
            except Exception as e:
                print("Le post_processing numéro " + str(num)
                      + " a levé une erreur.")
                print(e)

def coord_pattern_finder(image, template, threshold):
    """
    input:
        image : image plane cv2.imread() black and white
        template : image pattern cv2.imread()
        threshold : threshold for this pattern
    output:
        position : list right angle position for this pattern on the image

    """
    position = []  # Variable output
    # List of match
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    for pos in zip(*np.where(res >= threshold)[::-1]):
        position.append(pos)
    return(position)



In [ ]:
# SIEGE 


def coord_pattern_finder(image, template, threshold):
    """
    input:
        image : image plane cv2.imread() black and white
        template : image pattern cv2.imread()
        threshold : threshold for this pattern
    output:
        position : list right angle position for this pattern on the image

    """
    position = []  # Variable output
    # List of match
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    for pos in zip(*np.where(res >= threshold)[::-1]):
        position.append(pos)
    return(position)



In [ ]:
# from IPython.display import SVG
# import matplotlib.pyplot as plt
# plt.imread(SVG(url='https://cdn.seatguru.com/en_US/img/20200108203441/seatguru/airlines_new/Aer_Lingus/Aer_Lingus_Airbus_A330-200.svg'))

class NormalSeatNoCL(Process):
    process_desc = "OpenCV4.1.2.30 -> pattern matching for seat without colors or letters"

    def run(self, images: Iterable[Iterable]) -> None:
        position = []  # Variable output
        # List of match
        res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        for pos in zip(*np.where(res >= threshold)[::-1]):
            position.append(pos)
        return(position)
